### За основу взят код из https://www.kaggle.com/code/sunilgautam/xgboost-with-vgg16-for-image-classification и https://www.educative.io/answers/classification-using-xgboost-in-python

In [3]:
!jt -t grade3 -T -N -kl

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

import tensorflow as tf
# from tensorflow.keras.applications import VGG16

from PIL import Image
import cv2

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from skimage.io import imread
import xgboost

In [2]:
from IPython.display import display, HTML

display(HTML("<style>.container { width: 80% !important; }</style>"))

In [98]:
train_dir=r'C:\Users\79777\PycharmProjects\SmokersDetectionHack\Faces_new\train'
valid_dir=r'C:\Users\79777\PycharmProjects\SmokersDetectionHack\Faces_new\val'

### Создаём список с numpy массивами картинок тренировочного датасета и список лейблов

In [99]:
train_images = []
train_labels = []
IMG_SIZE = (56,56)
for folder,_,files in os.walk(train_dir):
    print(folder)
    for file in files:
        img = imread(os.path.join(folder,file))
        img = img[:,:,:3]
#         img = cv2.imread(os.path.join(folder,file), cv2.IMREAD_COLOR)
        img = cv2.resize(img, IMG_SIZE)
#         img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        label=0 if 'not_smoke' in folder else 1
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
print(train_labels)
print(len(train_images),len(train_labels))

C:\Users\79777\PycharmProjects\SmokersDetectionHack\Faces_new\train
C:\Users\79777\PycharmProjects\SmokersDetectionHack\Faces_new\train\not_smoke
C:\Users\79777\PycharmProjects\SmokersDetectionHack\Faces_new\train\smoke
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
57 57


### Создаём список с numpy массивами картинок тестового датасета и список лейблов. Также Сохраним имена тестовых изображений для тестирования обученной модели

In [100]:
valid_images = []
valid_labels = []
valid_names=[]
IMG_SIZE = (56,56)
for folder,_,files in os.walk(valid_dir):
    for file in files:
        img = imread(os.path.join(folder,file))
        img = img[:,:,:3]
#         img = cv2.imread(os.path.join(folder,file), cv2.IMREAD_COLOR)
        img = cv2.resize(img, IMG_SIZE)
#         img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        valid_images.append(img)
        label=0 if 'not_smoke' in folder else 1
        valid_labels.append(label)

valid_images = np.array(valid_images)
valid_labels = np.array(valid_labels)
print(valid_labels)
print(len(valid_images),len(valid_labels))

[0 0 0 0 0 0 0 0 0 0 0 0 1 1]
14 14


In [101]:
#Encode Labels
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)[:,None]
print(train_labels_encoded)
le.fit(valid_labels)
val_labels_encoded = le.transform(valid_labels)[:,None]
print(val_labels_encoded)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]]


In [110]:
# Загрузка предобученной свёрточной модели
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [112]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.DenseNet121(include_top=False,
                                                weights="imagenet",
                                                input_shape=IMG_SHAPE)

In [102]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.VGG19(include_top=False,
                                                weights="imagenet",
                                                input_shape=IMG_SHAPE)

In [113]:
# Посмотрим на архитектуру модели
base_model.trainable = False
base_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 56, 56, 3)]          0         []                            
                                                                                                  
 zero_padding2d_10 (ZeroPad  (None, 62, 62, 3)            0         ['input_15[0][0]']            
 ding2D)                                                                                          
                                                                                                  
 conv1/conv (Conv2D)         (None, 28, 28, 64)           9408      ['zero_padding2d_10[0][0]']   
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 28, 28, 64)           256       ['conv1/conv[0][0]']

In [104]:
train_features = base_model.predict(train_images)
train_features = train_features.reshape(train_features.shape[0], -1)
val_features = base_model.predict(valid_images)
val_features = val_features.reshape(val_features.shape[0], -1)

1/1 [==============================] - 0s 124ms/step


In [105]:
print(train_features.shape, train_labels_encoded.shape, val_features.shape, val_labels_encoded.shape)

(57, 512) (57, 1) (14, 512) (14, 1)


### save train and valid data

In [12]:
save_to=r'C:\Users\79777\Desktop\work\xgboost\experiment\23mm'
train_data=np.concatenate([train_features,train_labels_encoded],axis=1)
test_data=np.concatenate([val_features,val_labels_encoded],axis=1)
print(train_data.shape,test_data.shape)
np.save(os.path.join(save_to,'train_data.npy'),train_data)
np.save(os.path.join(save_to,'test_data.npy'),test_data)

(3677, 513) (1576, 513)


In [27]:
train_data.dtype

dtype('float64')

### XGBoost Classifier

In [106]:
model = xgboost.XGBClassifier(objective='binary:logistic',learning_rate=0.02,max_depth=70, n_estimators=600)
eval_set = [(train_features, train_labels_encoded), (val_features, val_labels_encoded)]

model.fit(
    train_features,
    train_labels_encoded
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=70, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=600, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [107]:
from sklearn.metrics import accuracy_score, classification_report
predictions = model.predict(val_features)

#Calculating accuracy
accuracy = accuracy_score(val_labels_encoded, predictions)

print(cl)
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(val_labels_encoded, predictions))

explosive_cord
Accuracy: 0.8571428571428571

Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92        12
           1       0.00      0.00      0.00         2

    accuracy                           0.86        14
   macro avg       0.43      0.50      0.46        14
weighted avg       0.73      0.86      0.79        14



C:\Users\79777\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\79777\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\79777\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [91]:
model.save_model(r"C:\Users\79777\PycharmProjects\SmokersDetectionHack\Wrists_new\wrist_model_densenet121.json")

In [33]:
model = xgboost.XGBClassifier()
model.load_model(r'C:\Users\79777\PycharmProjects\SmokersDetectionHack\Wrists_new\wrist_model.json')

In [14]:
y=np.concatenate([val_labels_encoded,predictions[:,None]],axis=1)
qnt0_0 = np.count_nonzero(np.all(y[:,:]==[0,0],axis=1))
qnt1_0 = np.count_nonzero(np.all(y[:,:]==[1,0],axis=1))
qnt0_1 = np.count_nonzero(np.all(y[:,:]==[0,1],axis=1))
qnt1_1 = np.count_nonzero(np.all(y[:,:]==[1,1],axis=1))
print(cl)
print(f'Всего {len(val_labels_encoded)} изображений в тестовом датасете\n'
      f'Всего {len(np.where(val_labels_encoded==0)[0])} реальных объектов\n'
      f'{qnt0_0} ({round(100*qnt0_0/len(np.where(val_labels_encoded==0)[0]),2)}%) реальных объектов распознаны как реальные\n'
      f'{qnt0_1} ({round(100*qnt0_1/len(np.where(val_labels_encoded==0)[0]),2)}%) реальных объектов распознаны как ложные\n'
      f'Всего {len(np.where(val_labels_encoded==1)[0])} ложных объектов\n'
      f'{qnt1_1} ({round(100*qnt1_1/len(np.where(val_labels_encoded==1)[0]),2)}%) ложных объектов распознаны как ложные\n'
      f'{qnt1_0} ({round(100*qnt1_0/len(np.where(val_labels_encoded==1)[0]),2)}%) ложных объектов распознаны как реальные\n')

explosive_cord
Всего 1608 изображений в тестовом датасете
Всего 1473 реальных объектов
1465 (99.46%) реальных объектов распознаны как реальные
8 (0.54%) реальных объектов распознаны как ложные
Всего 135 ложных объектов
79 (58.52%) ложных объектов распознаны как ложные
56 (41.48%) ложных объектов распознаны как реальные



### predict моделью xgboost картинок

In [39]:
# Скопируем картинки, которые XGboost классифицировал неверно в отдельные папки
import shutil
save_to=r'C:\Users\79777\Desktop\work\xgboost\experiment\explosive_cord\fake_predicted_as_real'
os.makedirs(save_to, exist_ok=True)
for ind in list(np.where(np.all(y[:,:]==[1,0],axis=1))[0]): # fake_predicted_as_real
    name=valid_names[ind]
    for folder,_,files in os.walk(valid_dir):
        for file in files:
            if file==name:
                shutil.copy(os.path.join(folder,file),os.path.join(save_to,file))
save_to=r'C:\Users\79777\Desktop\work\xgboost\experiment\explosive_cord\real_predicted_as_fake'
os.makedirs(save_to, exist_ok=True)
for ind in list(np.where(np.all(y[:,:]==[0,1],axis=1))[0]): # real_predicted_as_fake
    name=valid_names[ind]
    for folder,_,files in os.walk(valid_dir):
        for file in files:
            if file==name:
                shutil.copy(os.path.join(folder,file), os.path.join(save_to,file))
